In [1]:
!pip install newspaper3k

In [2]:
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python

Finished.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   657  100   657    0     0   1382      0 --:--:-- --:--:-- --:--:--  1386
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\vanhu\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vanhu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\vanhu\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\vanhu\AppData

In [3]:
from newspaper import Article
from bs4 import BeautifulSoup, Tag
import pandas as pd
import urllib.request
import requests
import os
import csv

In [6]:
soup = []
for i in range(1, 3): 
    if i == 1:
        link = 'https://vnexpress.net/kinh-doanh/chung-khoan'
    else:
        link = f'https://vnexpress.net/kinh-doanh/chung-khoan-p{i}'
    response = requests.get(link)
    soup.append(BeautifulSoup(response.content, "html.parser"))
    #titles = soup.find_all('section', attrs={'class':'section section_container mt15'})


In [7]:
links = []
# for s in soup:
#     for title in titles:    
#         items = s.find_all('h2', class_='title-news')
#         for item in items:
#             link = item.find('a', class_='')
#             if link and link.has_attr('href'):
#                 links.append(link.attrs["href"])
for element in soup:
    # Kiểm tra xem phần tử có phải là một Tag hợp lệ
    if isinstance(element, Tag):
        items = element.find_all('h2', class_='title-news')
        for item in items:
            link = item.find('a', class_='')
            if link and link.has_attr('href'):
                links.append(link.attrs["href"])
print(links)

['https://vnexpress.net/khoi-ngoai-ban-rong-phien-thu-6-lien-tiep-4685643.html', 'https://vnexpress.net/nha-dau-tu-gom-co-phieu-ldg-4685146.html', 'https://vnexpress.net/khoi-ngoai-quay-lung-voi-co-phieu-the-gioi-di-dong-4684781.html', 'https://vnexpress.net/lai-suat-trai-phieu-doanh-nghiep-van-nong-4684086.html', 'https://vnexpress.net/hon-39-trieu-co-phieu-ldg-treo-gia-san-4683637.html', 'https://vnexpress.net/vn-index-lai-giam-diem-o-nhung-phut-cuoi-4683207.html', 'https://vnexpress.net/them-doanh-nghiep-xin-khat-no-trai-phieu-4682829.html', 'https://vnexpress.net/vn-index-lay-lai-moc-1-100-diem-4682781.html', 'https://vnexpress.net/mot-sep-hah-ban-chui-co-phieu-vi-mat-kem-4682401.html', 'https://vnexpress.net/co-phieu-thep-nang-do-thi-truong-4682313.html', 'https://vnexpress.net/tien-vao-chung-khoan-giam-manh-4681866.html', 'https://vnexpress.net/cong-ty-bau-duc-huy-danh-sach-nha-dau-tu-sap-mua-130-trieu-co-phieu-4681125.html', 'https://vnexpress.net/chung-khoan-loi-nguoc-dong-tang

In [8]:
# Danh sách các từ khóa bạn quan tâm
keys = ["chứng khoán"]

# Hàm để kiểm tra xem một chuỗi có chứa bất kỳ từ khóa nào không
def contains_keyword(text, keywords):
    for keyword in keywords:
        if keyword in text.lower():
            return True
    return False

In [9]:
data = []
for link in links:
    news = requests.get(link)
    soup = BeautifulSoup(news.content, "html.parser")
    
     # Lấy tiêu đề
    title_tag = soup.find('h1', class_='title-detail')
    # title = title_tag.get_text().strip() if title_tag else None
    title = title_tag.get_text().strip() if title_tag else ""

    # Lấy tên tác giả
    author_tag = soup.find('strong')
    # author = author_tag.get_text().strip() if author_tag else None
    author = author_tag.get_text().strip() if author_tag else ""

    # Lấy thời gian đăng tải
    time_tag = soup.find('span', class_='date')
    # time = time_tag.get_text().strip() if time_tag else None
    if time_tag:
        time_full = time_tag.get_text().strip()
        # Chia chuỗi dựa vào dấu phẩy và lấy phần ngày tháng
        time_parts = time_full.split(',')
        time = time_parts[1].strip() if len(time_parts) > 1 else None
    else:
        time = None

    # Lấy mô tả
    description_tag = soup.find('p', class_='description')  
    # description = description_tag.get_text().strip() if description_tag else None
    description = description_tag.get_text().strip() if description_tag else ""
    
    content_divs = soup.find_all('article', class_='fck_detail ')  # This returns a ResultSet

    all_p_text = []
    for div in content_divs:
        p_tags = div.find_all('p', class_='Normal')
        for p in p_tags:
            text = p.get_text().strip()  
            if text:  
                all_p_text.append(text)
    content = ' '.join(all_p_text) if all_p_text else ""
    # article = Article(link, language='vi', request_timeout=10000)
    # article.download()
    # article.parse()
    # content = str(article.text).replace("\n","")
    
    # print(f"Title: {title}, Author: {author}, Time: {time}, Description: {description}, Link: {link}")#, Link: {link}
    if contains_keyword(title + " " + description + " " + content, keys):
        data.append([title, author, time, description, link])

for i in range(len(data)):
    url = data[i][-1]  # Giả sử URL là phần tử cuối cùng của mỗi hàng
    data[i][-1] = f'=HYPERLINK("{url}")'  # Tạo văn bản đặc biệt

directory = 'data/csv/vnexpress'
csv_file_path = f'{directory}/vnexpress.csv'
# Tạo thư mục nếu nó chưa tồn tại
if not os.path.exists(directory):
    os.makedirs(directory)

# Tạo DataFrame
df = pd.DataFrame(data, columns=['Title', 'Author', 'Time', 'Description', 'Link'])
    

In [10]:
# print(content)
# print(link)

In [11]:
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
print(f"Đã lưu dữ liệu vào file CSV: {csv_file_path}")

Đã lưu dữ liệu vào file CSV: data/csv/vnexpress/vnexpress.csv


In [12]:
# Hàm bạn đã cung cấp
def create_and_write_file(directory, filename, content):
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, filename)
    with open(file_path, 'w', encoding='utf-8-sig') as file:
        file.write(content)

keywords = ["chứng khoán"]

def contains_keyword(text):
    return any(keyword in text for keyword in keywords)

def crawl_by_url(url, title):
    article = Article(url, language='vi', request_timeout=10000)
    article.download()
    article.parse()
    contents = str(article.text).replace("\n","")
    article.nlp()
    # replace
    char_to_replace = {':': '',
                       '/': '',
                       '\\': '',
                      '*': '',
                      '?': '',
                      '"': '',
                      '<': '',
                      '>': '',
                      '|': ''}
    for key, value in char_to_replace.items():
        # Replace key character with value character in string
        title = title.replace(key, value)

    # write txt
    # https://vnexpress.net
    # if((url.find('https://vnexpress.net/') != -1)):
    #     directory = "data/text/vnexpress"
    #     output = title + ".txt"
    #     content = title + "\n" + contents

    # create_and_write_file(directory, output, content)
    
    if contains_keyword(title.lower()) or contains_keyword(contents.lower()):
        # Tiến hành lưu file nếu có từ khóa
        directory = "data/text/vnexpress"
        output = title + ".txt"
        content = title + "\n" + contents
        create_and_write_file(directory, output, content)

In [13]:
# VNEXPRESS
def get_links_in_page_vnexpress(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    # <section>
    sections = soup.find_all('section', attrs={'class':'section section_container mt15'})
    results = []
    links = []
    # <h3>
    h3_all = []
    for section in sections:
        h3_all.extend(section.find_all('h2', attrs={'class':'title-news'}))
    # <a>
    a_all = []
    results = [[]]
    for h3 in h3_all:
        a_all.extend(h3.find_all('a', attrs={'class':''}))
    for a in a_all:
        title = a.get('title')
        link = a.get('href')
        print(f'Title: {title} - Link: {link}')
        results.append([title,link])
        # links.append(link)
    return results

# base_url = 'https://vnexpress.net/kinh-doanh/chung-khoan'
# all_links = get_links_in_page_vnexpress(base_url)

# # Lưu vào danh sách links
# links = all_links[:2] 
# print(links)# Lấy 2 link đầu tiên
# get_links_in_page_vnexpress('https://vnexpress.net/kinh-doanh/chung-khoan')

In [11]:
# VNEXPRESS
def get_page_urls_vnexpress(base_url, quantity):
    news_per_page = 30
    if quantity % news_per_page == 0:
        no_of_pages = quantity // news_per_page
    else:
        no_of_pages = quantity // news_per_page + 1

    extend_url = ['-p{}'.format(page) for page in range(2, no_of_pages + 1)]
    url_list = [base_url + extend_part for extend_part in extend_url]
    url_list.insert(0, base_url)
    return url_list

In [12]:
# def vnexpress(base_url):
#     page_urls = get_page_urls_vnexpress(base_url, quantity=2)
# #     print(page_urls)
#     # total_articles = 0
#     all_data = []
#     for page in page_urls:
#         arr = get_links_in_page_vnexpress(page)
#         if arr:
#             all_data.extend(arr)
#         if arr is not None:
#             for i in range(1,len(arr)-1):
#                 title = arr[i][0]
#                 url = arr[i][1]
#                 crawl_by_url(url,title)
    
def vnexpress(base_url):
    page_urls = get_page_urls_vnexpress(base_url, quantity=2)
    total_articles = 0  # Biến đếm số lượng bài viết
    max_articles = 22  # Giới hạn số lượng bài viết tối đa

    for page in page_urls:
        if total_articles >= max_articles:
            break  # Dừng nếu đã đạt số lượng tối đa

        arr = get_links_in_page_vnexpress(page)
        if arr is not None:
            for i in range(1, len(arr) - 1):
                if total_articles >= max_articles:
                    break  # Kiểm tra lại trong vòng lặp

                title = arr[i][0]
                url = arr[i][1]
                crawl_by_url(url, title)
                total_articles += 1  # Tăng biến đếm

In [13]:
vnexpress('https://vnexpress.net/kinh-doanh/chung-khoan')

Title: Nhà đầu tư gom cổ phiếu LDG - Link: https://vnexpress.net/nha-dau-tu-gom-co-phieu-ldg-4685146.html
Title: Khối ngoại 'quay lưng' với cổ phiếu Thế Giới Di Động - Link: https://vnexpress.net/khoi-ngoai-quay-lung-voi-co-phieu-the-gioi-di-dong-4684781.html
Title: Lãi suất trái phiếu doanh nghiệp vẫn nóng - Link: https://vnexpress.net/lai-suat-trai-phieu-doanh-nghiep-van-nong-4684086.html
Title: Hơn 39 triệu cổ phiếu LDG treo giá sàn - Link: https://vnexpress.net/hon-39-trieu-co-phieu-ldg-treo-gia-san-4683637.html
Title: VN-Index lại giảm điểm ở những phút cuối - Link: https://vnexpress.net/vn-index-lai-giam-diem-o-nhung-phut-cuoi-4683207.html
Title: Thêm doanh nghiệp xin khất nợ trái phiếu - Link: https://vnexpress.net/them-doanh-nghiep-xin-khat-no-trai-phieu-4682829.html
Title: VN-Index lấy lại mốc 1.100 điểm - Link: https://vnexpress.net/vn-index-lay-lai-moc-1-100-diem-4682781.html
Title: Một sếp HAH bán chui cổ phiếu 'vì mắt kém' - Link: https://vnexpress.net/mot-sep-hah-ban-chui

In [14]:
path = r"C:\Users\vanhu\Desktop\Crawlnews\crawling-VietNam-News\data\text\vnexpress"
import os

def count_txt_files(directory):
    # Đảm bảo thư mục tồn tại
    if not os.path.exists(directory):
        print("Thư mục không tồn tại.")
        return 0

    # Đếm số file .txt
    count = 0
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            count += 1

    return count

# Ví dụ sử dụng
num_txt_files = count_txt_files(path)
print(f"Có {num_txt_files} file .txt trong thư mục {path}.")


Có 21 file .txt trong thư mục C:\Users\vanhu\Desktop\Crawlnews\crawling-VietNam-News\data\text\vnexpress.
